# **Scrape**

Site to scrape: https://statewidedatabase.org/d10/p16.html
        
table, tbody, -- <br>
    go through every row (tr) <br>
    go through every header (th) in row <br>
    get th > a:text <br>
    get everything inside th <br>


In [1]:
# Dependencies
from bs4 import BeautifulSoup
import pymongo
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd


# Dependencies
# ----------------------------------
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from config import ADDRESS,PORTNUM,USERNAME,PW,DBNAMEPC
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session

-------

**PostgreSQL Auth**

Create the connection

In [2]:
# Postgres username, password, and database name
POSTGRES_ADDRESS = ADDRESS ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = PORTNUM
POSTGRES_USERNAME = USERNAME ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = PW ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD
POSTGRES_DBNAME = DBNAMEPC ## CHANGE THIS TO YOUR DATABASE NAME

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
    .format(username=POSTGRES_USERNAME,
    password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DBNAME))


-----

**Splinter**

In [3]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url = 'https://statewidedatabase.org/d10/p16.html'
browser.visit(url)

-----

In [5]:
# HTML object
html = browser.html
# Instantiate a BeautifulSoup() object with our `html` and the `html5lib` parser
soup = BeautifulSoup(html, 'html5lib')

# Article holding pics
tbody = soup.find('tbody')

In [6]:
# tr holds all a:text which also holds county number
tr = tbody.find_all('a', class_='county-num')

In [8]:
# Empty list for loop
county_number_list = []

# Find County Number
for t in tr:
    # County number: County 001
    county_number = t.next_element
    
    # initializing sub list. Remove the str 'COUNTY ' 
    sub_list = ["COUNTY "] 
    
    # For item in list, remove string found in sub_list and replace with what is left over.
    for sub in sub_list: 
        county_str = county_number.replace(sub, ' ') 
        res = " ".join(county_str.split())
    
    # Add results to list
    county_number_list.append(res)
# When finished, print list
# print(county_number_list)

In [9]:
# tbody holds all thead which has city name. county number is also here but within anchor text
tbody = soup.find('tbody')
th = tbody.find_all('th')

# Empty name list for loop
county_name_list = []

# Loop through headers and grab city name while avoiding anchor
for t in th:
    th = t.next_element.next_element.next_element.next_element
    county_name_list.append(th)

# Scraping included a row we didnt need. Removing.
remove_first_item_in_list = list(county_name_list.pop(0))
# print(county_name_list)
browser.quit()

In [10]:
# Combine list to dict for dataframe cleaning
county_num_name_dict = dict(zip(county_number_list, county_name_list))

Create dataframe for viewing

In [11]:
df_county_name_num = pd.DataFrame(columns=['county', 'county_name'])
df_county_name_num['county'] = county_num_name_dict.keys()
df_county_name_num['county_name'] = county_num_name_dict.values()
# df_county_name_num

In [12]:
# Check if all 58 counties are listed
assert len(df_county_name_num) == 58;
print('Good for upload to db.')

Good for upload to db.


------

# Upload to PostgreSQL

In [13]:
# TEMPLATE: CountNamesNumber template to upload to specific table in db
# Create CountNamesNumber Classes
# Creates table with column names
# ----------------------------------
class CountNamesNumber(Base):
    __tablename__ = 'county_names'
    county = Column(Integer, primary_key=True)
    county_name = Column(String(30))
    

In [14]:
# Create Database Connection
# ----------------------------------
# create engine to postgres connection
engine = create_engine(postgres_str)
conn = engine.connect()

In [15]:
# Create a "Metadata" Layer That Abstracts the SQL Database
# ----------------------------------
Base.metadata.create_all(engine)

In [16]:
# Create a Session Object to Connect to DB
# ----------------------------------
session = Session(bind=engine)

Loop through both list created. Assign value to be added individually to specified column

In [17]:
try:
    for key, value in zip(county_number_list, county_name_list):
    #     print(f'{key},{value}')
        row = CountNamesNumber(county=key, county_name=value)
        session.add(row)
        session.commit()
    print('completed upload to db')
except:
    print('error during upload. check db for partial information.')

completed upload to db
